# Pricing Model using Deep Learning

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical

print(tf.__version__)

2.1.0


In [28]:
listings=pd.read_csv('listings.csv')
print('El dataset de entrada contiene', len(listings.columns), 'columnas', 'y', len(listings), 'filas')

El dataset de entrada contiene 92 columnas y 3818 filas


In [29]:
for col in listings.columns: ##Eliminando las columnas que tienen un solo valor 
    if len(listings[col].unique()) == 1:
        listings.drop(col,inplace=True,axis=1)
        
print('Luego de remover las columnas con un valor único permanecen', len(listings.columns),'columnas')

Luego de remover las columnas con un valor único permanecen 81 columnas


In [30]:
def convert_to_numbers(dataset, column):
    dataset[column]=dataset[column].str.replace('[$,]', '').astype('float')
    return print('La columna', column, 'ha sido modificada')

Columns_to_convert=['price', 'cleaning_fee', 'extra_people', 'security_deposit']

for variable in Columns_to_convert:
    convert_to_numbers(listings, variable)


La columna price ha sido modificada
La columna cleaning_fee ha sido modificada
La columna extra_people ha sido modificada
La columna security_deposit ha sido modificada


In [31]:
def removing_percentage (dataset, column):
    dataset[column]=dataset[column].str.replace('%', '').astype('float')
    print('La columna', column, 'ha sido modificada')

removing_perc=['host_acceptance_rate', 'host_response_rate']

for var in removing_perc:
    removing_percentage(listings, var)

La columna host_acceptance_rate ha sido modificada
La columna host_response_rate ha sido modificada


In [32]:
listings.cleaning_fee=listings.cleaning_fee.fillna(0) ##Asumiendo que los vacios de estos campos corresponden a 0
listings.security_deposit=listings.security_deposit.fillna(0)

In [33]:
most_missing_cols = set(listings.columns[listings.isnull().mean() > 0.30])
most_missing_cols #Columnas con un porcentaje de datos faltantes superior al 30%

{'monthly_price', 'notes', 'square_feet', 'weekly_price'}

In [34]:
url_field=list(listings.filter(like='url').columns)
other_pricing_field=list(listings.filter(like='_price').columns)
text_field=['notes','neighborhood_overview', 'host_about','transit', 'space','summary','description']
other_geo=[ 'zipcode', 'neighbourhood', 'host_name','smart_location',  'is_location_exact']
availability=['availability_60', 'availability_90', 'availability_365']
host=['host_verifications', 'host_total_listings_count' ,'host_has_profile_pic', 'host_location', 'host_id','host_neighbourhood','host_location']

In [35]:
listings=listings.drop(url_field+other_pricing_field+text_field+other_geo+availability+host+['square_feet','first_review', 'last_review'], axis=1)

In [36]:
polarity=pd.read_csv('reviews_polarity.csv')
listings=listings.merge(polarity, left_on='id', right_on='listing_id', how='left')
listings.columns

Index(['id', 'name', 'host_since', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_identity_verified', 'street', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'bed_type', 'amenities', 'price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'calendar_updated',
       'availability_30', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'reviews_

In [38]:
host_response_time = listings.pop('host_response_time')

In [39]:
host_response_time.unique()  ##Debo convertirlos en categorias numericas y luego en dummies

array(['within a few hours', 'within an hour', nan, 'within a day',
       'a few days or more'], dtype=object)

In [40]:
#response_time_cat=pd.DataFrame(to_categorical(host_response_time))

In [41]:
listings.pivot_table(index='host_identity_verified', values='id', aggfunc='count')

,id
host_identity_verified,
f,819
t,2997


In [42]:
listings.host_since

0       2011-08-11
1       2013-02-21
2       2014-06-12
3       2013-11-06
4       2011-11-29
           ...    
3813    2015-04-13
3814    2015-10-14
3815    2015-12-30
3816    2015-01-03
3817    2014-04-25
Name: host_since, Length: 3818, dtype: object

In [43]:
listings.pivot_table(index='host_is_superhost', values=['id'], aggfunc='count')

,id
host_is_superhost,
f,3038
t,778


In [44]:
listings.pivot_table(index='cancellation_policy', values='id', aggfunc='count')

,id
cancellation_policy,
flexible,1150
moderate,1251
strict,1417


In [45]:
listings.pivot_table(index='property_type', values=['id'], aggfunc='count')

,id
property_type,
Apartment,1708
Bed & Breakfast,37
Boat,8
Bungalow,13
Cabin,21
Camper/RV,13
Chalet,2
Condominium,91
Dorm,2


In [46]:
listings.pivot_table(index= 'calculated_host_listings_count', values='id', aggfunc='count')

,id
calculated_host_listings_count,
1,2451
2,575
3,262
4,130
5,70
6,65
7,21
8,19
9,19


In [47]:
listings.dropna()

,id,name,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_identity_verified,street,neighbourhood_cleansed,...,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,listing_id,comments_polarity
0,241032,Stylish Queen Anne Apartment,2011-08-11,96.0,100.0,f,3.0,t,"Gilman Dr W, Seattle, WA 98119, United States",West Queen Anne,...,9.0,10.0,f,moderate,f,f,2,4.07,241032.0,0.377606
1,953595,Bright & Airy Queen Anne Apartment,2013-02-21,98.0,100.0,t,6.0,t,"7th Avenue West, Seattle, WA 98119, United States",West Queen Anne,...,10.0,10.0,f,strict,t,t,6,1.48,953595.0,0.399665
2,3308979,New Modern House-Amazing water view,2014-06-12,67.0,100.0,f,2.0,t,"West Lee Street, Seattle, WA 98119, United States",West Queen Anne,...,10.0,10.0,f,strict,f,f,2,1.15,3308979.0,0.468537
6,1909058,Queen Anne Private Bed and Bath,2012-05-30,100.0,100.0,t,1.0,t,"10th Avenue West, Seattle, WA 98119, United St...",West Queen Anne,...,10.0,10.0,f,moderate,f,f,1,2.46,1909058.0,0.346848
7,856550,Tiny Garden cabin on Queen Anne,2012-10-30,100.0,100.0,t,5.0,t,"West Wheeler Street, Seattle, WA 98119, United...",West Queen Anne,...,9.0,10.0,f,strict,t,t,5,4.73,856550.0,0.354141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3810,262764,Fremont Farmhouse,2011-11-07,100.0,100.0,f,1.0,t,"N Bowdoin Pl, Seattle, WA 98103, United States",Fremont,...,10.0,8.0,f,strict,f,f,1,1.56,262764.0,0.276053
3811,8578490,Super Convenient Top Floor Apt,2014-02-01,100.0,100.0,f,1.0,t,"Whitman Avenue North, Seattle, WA 98103, Unite...",Fremont,...,10.0,10.0,f,moderate,f,f,1,0.63,8578490.0,0.353000
3812,3383329,OF THE TREE & CLOUDS. KID FRIENDLY!,2012-09-01,100.0,100.0,t,3.0,t,"Northwest 39th Street, Seattle, WA 98107, Unit...",Fremont,...,9.0,10.0,t,moderate,t,t,3,4.01,3383329.0,0.400162
3813,8101950,3BR Mountain View House in Seattle,2015-04-13,99.0,100.0,f,354.0,t,"Northwest 48th Street, Seattle, WA 98107, Unit...",Fremont,...,10.0,8.0,f,strict,f,f,8,0.30,8101950.0,0.000000


In [48]:
2655/3800

0.6986842105263158

In [49]:
listings.amenities

0       {TV,"Cable TV",Internet,"Wireless Internet","A...
1       {TV,Internet,"Wireless Internet",Kitchen,"Free...
2       {TV,"Cable TV",Internet,"Wireless Internet","A...
3       {Internet,"Wireless Internet",Kitchen,"Indoor ...
4       {TV,"Cable TV",Internet,"Wireless Internet",Ki...
                              ...                        
3813    {TV,"Cable TV",Internet,"Wireless Internet","A...
3814    {TV,"Cable TV",Internet,"Wireless Internet",Ki...
3815    {"Cable TV","Wireless Internet",Kitchen,"Free ...
3816    {TV,"Wireless Internet",Kitchen,"Free Parking ...
3817    {TV,"Cable TV",Internet,"Wireless Internet",Ki...
Name: amenities, Length: 3818, dtype: object

In [50]:
listings.extra_people

0        5.0
1        0.0
2       25.0
3        0.0
4       15.0
        ... 
3813     0.0
3814    25.0
3815    20.0
3816     0.0
3817     0.0
Name: extra_people, Length: 3818, dtype: float64